In [1]:
import numpy             as np
import matplotlib.pyplot as plt
import pandas            as pd
import pygimli           as pg
import pygimli.meshtools as mt
import cmasher           as cmr
import pyvista           as pyv
from pygimli.viewer             import pv

# to convert to correct coordinates
[x, y]        = [2549161.271, 5569492.241]

In [12]:
# Load files

fn_dir = 'C:/Users/azieg/Desktop/GIT_repositories/APG-MSc-Project-Ziegon/Data/old_Model/'

nodes = np.loadtxt(fn_dir+'Nodes.txt', comments="#", delimiter=" ", unpack=False)
nodes[:,:2] = -nodes[:,:2]


cells = np.loadtxt(fn_dir+'Cells.txt', comments="#", delimiter=" ", unpack=False, dtype=int)[:,1:]
marks = np.loadtxt(fn_dir+'Markers.txt', comments="#", delimiter=" ", unpack=False, dtype=int)

In [13]:
mp = [0.5*(np.max(nodes[:,0])-np.min(nodes[:,0])), 0.5*(np.max(nodes[:,1])-np.min(nodes[:,1]))]

In [14]:
def rotate(origin, point, angle):
    """
    Rotate a point counterclockwise by a given angle around a given origin.

    The angle should be given in radians.
    """
    ox, oy = origin
    px, py = point

    qx = ox + np.cos(angle) * (px - ox) - np.sin(angle) * (py - oy)
    qy = oy + np.sin(angle) * (px - ox) + np.cos(angle) * (py - oy)
    return qx, qy

In [15]:
nodes[:,0] += x
nodes[:,1] += y

# idices of nodes that are edge nodes and not 'in' diatreme
no_idx = np.array([n[0] for n in np.argwhere(nodes[:,0]<(-120+x))] + [n[0] for n in np.argwhere(nodes[:,0]>(-20+x))])

# Rotate accordingly to Topography script
pos_new = []
for p in nodes:
    x_new, y_new = rotate([x,y], [p[0],p[1]], np.deg2rad(11))
    pos_new.append([x_new,y_new,p[2]])

nodes = np.array(pos_new)

In [16]:
dia = pg.Mesh(3)
print(dia)

# Nodes
for n in nodes:
    dia.createNode((n[0],n[1],n[2]))
print(dia)

# Mantle
for i in range(len(marks)):
    if marks[i]==2:
        
        n1 = dia.nodes()[cells[i,0]]
        n2 = dia.nodes()[cells[i,1]]
        n3 = dia.nodes()[cells[i,2]]
        if cells[i,0] not in no_idx and cells[i,1] not in no_idx and cells[i,2] not in no_idx:
            dia.createTriangle(n1, n2, n3, marker=2)
print(dia)

# Boundaries
dia.createNeighborInfos()
print(dia)

Mesh: Nodes: 0 Cells: 0 Boundaries: 0
Mesh: Nodes: 1119 Cells: 0 Boundaries: 0
Mesh: Nodes: 1119 Cells: 108 Boundaries: 0
Mesh: Nodes: 1119 Cells: 108 Boundaries: 196


In [17]:
%matplotlib qt
pl, _ = pg.show(dia, hold=True, notebook=False, alpha=0.5, cMap=cmr.tropical)

pv.drawSensors(pl, np.array([[mp[0]+x,mp[1]+y,450],[70+x,60+y,380],[90+x,60+y,433]]), diam=5, color='red')

pl.show()

## Load 3d mesh

In [18]:
fn_dir  = 'C:/Users/azieg/Desktop/GIT_repositories/APG-MSc-Project-Ziegon/Data/Rockeskyll/'
mesh = pg.load(fn_dir+'invmesh.bms')

res_jme1 = np.load('C:/Users/azieg/Desktop/GIT_repositories/APG-MSc-Project-Ziegon/Data/Rockeskyll/Res_JME1_4/res_jme.npy')
sus_jme1 = np.load('C:/Users/azieg/Desktop/GIT_repositories/APG-MSc-Project-Ziegon/Data/Rockeskyll/Res_JME1_4/sus_jme.npy')

res_lim= res_jme1.copy()

for i, r in enumerate(res_lim):
    if r<20:
        res_lim[i] = 20
    elif r>400:
        res_lim[i] = 400

mesh['sus'] = sus_jme1
mesh['res'] = res_lim

In [21]:
pl, _ = pg.show(mesh,pg.z(mesh.cellCenters()), cMap='terrain', hold=True, notebook=False, alpha=0.5)
pv.drawMesh(pl, dia, cMap=cmr.tropical)
pv.drawMesh(pl, mesh, label="sus", style="surface", cMap=cmr.tropical,
            filter={"threshold": dict(value=0.11, scalars="sus")})
pv.drawSensors(pl, np.array([[x,y,440]]), diam=5, color='red')
pl.show()

In [22]:
dia.exportVTK('Diatreme_Model')